### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 1, 21)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-21
yesterday: 2025-01-20


In [5]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [9]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Datasets path (dts_path): {dts_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"OSD path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Datasets path (dts_path): C:\Users\PC1\Downloads\Datasets
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
OSD path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [11]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-01-20'
ORDER BY a.name



(20, 1)

In [13]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-20' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [16]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-14,2025-01-20,5.10,5.15,25049174,5.15,5.05,B,SET,0.05,0.98
1,AIMIRT,2025-01-13,2025-01-20,10.50,10.50,2869400,10.60,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-01-17,2025-01-20,12.20,11.90,2155940,12.20,11.90,B,SET,-0.30,-2.46
3,DIF,2025-01-16,2025-01-20,8.25,8.35,24829964,8.35,8.20,B,SET,0.10,1.21
4,GVREIT,2025-01-20,2025-01-20,6.65,6.55,1763967,6.65,6.45,B,SET,-0.10,-1.50
5,JMART,2025-01-16,2025-01-20,11.20,10.70,17815986,11.80,10.70,B,SET50,-0.50,-4.46
6,KCE,2025-01-17,2025-01-20,21.70,22.00,13118570,22.50,21.60,B,SET100,0.30,1.38
7,MCS,2025-01-08,2025-01-20,6.45,6.30,2104686,6.60,6.30,B,SET,-0.15,-2.33
8,NER,2025-01-17,2025-01-20,4.84,4.94,16351458,4.94,4.82,B,SET,0.10,2.07
9,SENA,2025-01-09,2025-01-20,2.20,2.18,3126720,2.24,2.16,B,SET,-0.02,-0.91


In [18]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [22]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-14,2025-01-20,5.10,5.15,"25,049,174",5.15,5.05,0.98%,B
1,AIMIRT,2025-01-13,2025-01-20,10.50,10.50,"2,869,400",10.60,10.40,0.00%,B
2,CPNREIT,2025-01-17,2025-01-20,12.20,11.90,"2,155,940",12.20,11.90,-2.46%,B
3,DIF,2025-01-16,2025-01-20,8.25,8.35,"24,829,964",8.35,8.20,1.21%,B
4,GVREIT,2025-01-20,2025-01-20,6.65,6.55,"1,763,967",6.65,6.45,-1.50%,B
5,JMART,2025-01-16,2025-01-20,11.20,10.70,"17,815,986",11.80,10.70,-4.46%,B
6,KCE,2025-01-17,2025-01-20,21.70,22.00,"13,118,570",22.50,21.60,1.38%,B
7,MCS,2025-01-08,2025-01-20,6.45,6.30,"2,104,686",6.60,6.30,-2.33%,B
8,NER,2025-01-17,2025-01-20,4.84,4.94,"16,351,458",4.94,4.82,2.07%,B
9,SENA,2025-01-09,2025-01-20,2.20,2.18,"3,126,720",2.24,2.16,-0.91%,B


In [24]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"output_file (output_file): {output_file}") 
print(f"icd_file (icd_file): {icd_file}") 
print(f"god_file (god_file): {god_file}") 
print(f"osd_file (osd_file): {osd_file}") 

output_file (output_file): C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file (icd_file): C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file (god_file): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file (osd_file): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [26]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [30]:
sales = df[cols]
sales.shape

(20, 10)

In [32]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
prices = pd.read_csv(input_file)
prices.shape

(181, 10)

In [34]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [36]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-14,2025-01-20,5.10,5.15,25049174,5.15,5.05,0.98,B,5.10,-0.05,10016093,2025-01-21
1,AIMIRT,2025-01-13,2025-01-20,10.50,10.50,2869400,10.60,10.40,0.00,B,10.50,0.00,519287,2025-01-21
2,CPNREIT,2025-01-17,2025-01-20,12.20,11.90,2155940,12.20,11.90,-2.46,B,12.10,0.20,1721910,2025-01-21
3,DIF,2025-01-16,2025-01-20,8.25,8.35,24829964,8.35,8.20,1.21,B,8.45,0.10,3198197,2025-01-21
4,GVREIT,2025-01-20,2025-01-20,6.65,6.55,1763967,6.65,6.45,-1.50,B,6.55,0.00,496200,2025-01-21
5,JMART,2025-01-16,2025-01-20,11.20,10.70,17815986,11.80,10.70,-4.46,B,11.20,0.50,3769709,2025-01-21
6,KCE,2025-01-17,2025-01-20,21.70,22.00,13118570,22.50,21.60,1.38,B,23.20,1.20,11324881,2025-01-21
7,MCS,2025-01-08,2025-01-20,6.45,6.30,2104686,6.60,6.30,-2.33,B,6.35,0.05,33401,2025-01-21
8,NER,2025-01-17,2025-01-20,4.84,4.94,16351458,4.94,4.82,2.07,B,4.94,0.00,4939077,2025-01-21
9,SENA,2025-01-09,2025-01-20,2.20,2.18,3126720,2.24,2.16,-0.91,B,2.18,0.00,175800,2025-01-21


In [38]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [40]:
print(f"output_file (output_file): {output_file}") 
print(f"icd_file (icd_file): {icd_file}") 
print(f"god_file (god_file): {god_file}") 
print(f"osd_file (osd_file): {osd_file}") 

output_file (output_file): C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file (icd_file): C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file (god_file): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file (osd_file): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [42]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [44]:
# Get the current working directory
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

Current Directory: C:\Users\PC1\OneDrive\A5\Daily


In [46]:
# Derive the base directory (to_dir) by removing the last folder ('Daily')
to_dir = os.path.dirname(current_dir)
print(f"Base Directory (to_dir): {to_dir}")

Base Directory (to_dir): C:\Users\PC1\OneDrive\A5


In [48]:
os.chdir(to_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [50]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-21 2025-01-21   5.15   5.10  -0.97%  25,049,174   5.15   5.05 B Insert
AIMIRT   2025-01-13 2025-01-21  10.50  10.50    0.0%   2,869,400  10.60  10.40 B Update
AWC      2025-01-21 2025-01-21   3.04   3.10   1.97% 111,757,948   3.18   3.04 S Insert
BCH      2025-01-21 2025-01-21  14.20  14.50   2.11%  11,559,696  14.90  14.20 S Insert
CPNREIT  2025-01-21 2025-01-21  11.90  12.10   1.68%   2,155,940  12.20  11.90 B Insert
DIF      2025-01-16 2025-01-21   8.25   8.45   2.42%  24,829,964   8.45   8.20 B Update
GVREIT   2025-01-20 2025-01-21   6.65   6.55   -1.5%   1,763,967   6.65   6.45 B Update
JMART    2025-01-21 2025-01-21  10.70  11.20   4.67%  17,815,986  11.80  10.70 B Insert
JMT      2025-01-21 2025-01-21  14.30  14.90    4.2%  72,617,738  15.70  14.30 S Insert
KCE      2025-01-17 2025-01-21  

In [52]:
os.chdir(current_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [54]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-21' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [56]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [58]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-21,2025-01-21,5.15,5.10,"25,049,174",5.15,5.05,B,SET,-0.05,-0.97%
1,AIMIRT,2025-01-13,2025-01-21,10.50,10.50,"5,738,800",10.60,10.40,B,SET,0.00,0.00%
2,CPNREIT,2025-01-21,2025-01-21,11.90,12.10,"2,155,940",12.20,11.90,B,SET,0.20,1.68%
3,DIF,2025-01-16,2025-01-21,8.25,8.45,"49,659,928",8.45,8.20,B,SET,0.20,2.42%
4,GVREIT,2025-01-20,2025-01-21,6.65,6.55,"3,527,934",6.65,6.45,B,SET,-0.10,-1.50%
5,JMART,2025-01-21,2025-01-21,10.70,11.20,"17,815,986",11.80,10.70,B,SET50,0.50,4.67%
6,KCE,2025-01-17,2025-01-21,21.70,23.20,"26,237,140",23.20,21.60,B,SET100,1.50,6.91%
7,MCS,2025-01-21,2025-01-21,6.30,6.35,"2,104,686",6.60,6.30,B,SET,0.05,0.79%
8,NER,2025-01-17,2025-01-21,4.84,4.94,"32,702,916",4.94,4.82,B,SET,0.10,2.07%
9,SENA,2025-01-09,2025-01-21,2.20,2.18,"6,253,440",2.24,2.16,B,SET,-0.02,-0.91%


In [60]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [62]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
18,TVO,2025-01-02,2025-01-21,22.90,21.10,"15,608,732",23.20,21.10,S,SET,-1.80,-7.86%


In [64]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
6,KCE,2025-01-17,2025-01-21,21.70,23.20,"26,237,140",23.20,21.60,B,SET100,1.50,6.91%


In [66]:
conlite.close()